In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# https://www.kaggle.com/fedesoriano/heart-failure-prediction

df = pd.read_csv('heart.csv')
df.head()
df.shape

(918, 12)

In [3]:

test_df = df[df.FastingBS>(df.FastingBS.mean()+3*df.FastingBS.std())]
len(test_df)

0

In [4]:
test_df  = df[df.Oldpeak>(df.Oldpeak.mean()+3*df.Oldpeak.std())]
len(test_df)

6

#### Remove Outliers

In [5]:
def Remove_Outliers(df, field):
    mean = df[field].mean()
    std = df[field].std()
    threshold = mean + 3 * std
    
    # Filter rows where the field value exceeds the threshold
    outliers = df[df[field] > threshold]
    
    if len(outliers) == 0:
        print(f"No outliers found for {field}")
        return df
    else:
        df_cleaned = df[df[field] <= threshold]
        print(f"After removing outliers for {field}: {df_cleaned.shape}")
        return df_cleaned

list_fields = ['RestingBP', 'Cholesterol','FastingBS','MaxHR','Oldpeak']

for i in list_fields:
    df = Remove_Outliers(df, i)

print("Final dataframe after removing all the outliers has following shape", df.shape)

After removing outliers for RestingBP: (911, 12)
After removing outliers for Cholesterol: (908, 12)
No outliers found for FastingBS
No outliers found for MaxHR
After removing outliers for Oldpeak: (902, 12)
Final dataframe after removing all the outliers has following shape (902, 12)


In [6]:
df.shape

(902, 12)

In [7]:
df.ChestPainType.unique()

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [8]:
df4.ExerciseAngina.unique()

NameError: name 'df4' is not defined

In [ ]:
df4.ST_Slope.unique()

In [12]:
df4 = df.copy()
df4.ExerciseAngina.replace(
    {
        'N': 0,
        'Y': 1
    },
    inplace=True)

df4.ST_Slope.replace(
    {
        'Down': 1,
        'Flat': 2,
        'Up': 3
    },
    inplace=True
)

df4.RestingECG.replace(
    {
        'Normal': 1,
        'ST': 2,
        'LVH': 3
    },
    inplace=True)



df4.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,1,172,0,0.0,3,0
1,49,F,NAP,160,180,0,1,156,0,1.0,2,1
2,37,M,ATA,130,283,0,2,98,0,0.0,3,0
3,48,F,ASY,138,214,0,1,108,1,1.5,2,1
4,54,M,NAP,150,195,0,1,122,0,0.0,3,0


In [13]:
df5 = pd.get_dummies(df4, drop_first=True).astype(int)
df5.head()

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,140,289,0,1,172,0,0,3,0,1,1,0,0
1,49,160,180,0,1,156,0,1,2,1,0,0,1,0
2,37,130,283,0,2,98,0,0,3,0,1,1,0,0
3,48,138,214,0,1,108,1,1,2,1,0,0,0,0
4,54,150,195,0,1,122,0,0,3,0,1,0,1,0


In [14]:
X= df5.drop(['HeartDisease'], axis = 'columns')
Y = df5.HeartDisease

In [15]:
from sklearn.preprocessing import StandardScaler
scale =  StandardScaler()
scaled_X = scale.fit_transform(X)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_X, Y, test_size=0.33, random_state=42)

In [18]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators= 20)
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=20)

In [19]:
clf.score(X_test, y_test)

0.8489932885906041

## Use PCA

In [22]:
from sklearn.decomposition import PCA
clf_pca =  PCA(0.95)
X_pca = clf_pca.fit_transform(X)

In [23]:
from sklearn.model_selection import train_test_split
X_pca_train, X_pca_test, y_pca_train, y_pca_test = train_test_split(X_pca, Y, test_size=0.33, random_state=42)

In [24]:
clf_pca = RandomForestClassifier(n_estimators= 5)
clf_pca.fit(X_pca_train, y_pca_train)

RandomForestClassifier(n_estimators=5)

In [25]:
clf_pca.score(X_pca_test,y_pca_test)

0.6845637583892618

### Use CrossValidation technique


In [50]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

scores = cross_val_score(DecisionTreeClassifier(random_state=0),scaled_X, Y)
scores.mean()

0.7260405156537753

In [48]:
scores = cross_val_score(RandomForestClassifier(random_state=1),scaled_X, Y)
scores.mean()

0.8203621853898098

In [57]:
scores = cross_val_score(SVC(), scaled_X,Y)
scores.mean()

0.8148004910988336

### Use Bagging Classifier technique


In [56]:
from sklearn.ensemble import BaggingClassifier
bag_model = BaggingClassifier(SVC(), n_estimators=100, max_samples=0.8, random_state=0)
scores = cross_val_score(bag_model, scaled_X,Y , cv=5)
scores.mean()


0.8170165745856354

***As you can see above, using bagging in case of SVM doesn't make much difference in terms of model accuracy. Bagging is effective when we have high variance and instable model such as decision tree. Let's explore how bagging changes the performance for a decision tree classifier.***


***Clearly through Bagging the score for decision tree increased from 72% to 84%***

In [59]:
bag_model = BaggingClassifier(RandomForestClassifier(random_state = 1),n_estimators=100, max_samples=10, oob_score= True,random_state= 1 )
score = cross_val_score(bag_model,scaled_X,Y , cv=5)
score.mean()


0.8258870472682629

**Random forest gave even a better performance with 81.7% as score. Underneath it used bagging where it sampled not only data rows but also the columns (or features)**